In [180]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random as rd

In [181]:
""" 
 Only for 1D input 
 
"""


class RNN: 
    def __init__(self, start_random):
        self.Whx = None
        self.Whh = None
        self.Why = None 
        self.bh = None 
        self.by = None
        
        self.activation = self.sigmoid
        self.d_activation = self.d_sigmoid
        
        self.start_random = start_random
    
    def init_weights(self, whx=0, whh=0, why=0, bh = 0, by = 0):    
        self.Whx = 0 if not self.start_random else rd.random()
        self.Whh = 0 if not self.start_random else rd.random()
        self.Why = 0 if not self.start_random else rd.random()
    
        self.bh = bh
        self.by = by
    
        return self
    
    def mean_squared_error(self, true, pred):
        return np.mean((true - pred)**2)
    
    def sigmoid(self, x):
        return 1/(1 + np.exp(-x)) 
    
    def d_sigmoid(self,x ):
        return self.sigmoid(x) * (1 - self.sigmoid(x))
    
    def train(self, x, y, learning_rate=0.01, epochs=100):
        if self.Whx is None:
            self.init_weights()

        for epoch in range(epochs):
            total_loss = 0
            for i in range(len(x)):
                # Forward pass
                output, states = self.forward_pass(x[i], get_states=True)

                # Backward pass
                loss = self.mean_squared_error(y[i], output)
                total_loss += loss
z
                # Initialize gradients
                d_Whx = 0
                d_Whh = 0
                d_Why = 0
                d_bh = 0
                d_by = 0

                # Compute gradients for each time step in the sequence
                d_output = output - y[i]
                for t in range(len(x[i])):
                    d_hidden = d_output * self.Why * self.d_activation(states[t])

                    # Accumulate gradients
                    d_Whx += d_hidden * x[i][t]
                    d_Whh += d_hidden * states[t]
                    d_Why += d_output * self.activation(states[t])
                    d_bh += d_hidden
                    d_by += d_output

                # Update weights and biases after the entire sequence
                self.Whx -= learning_rate * d_Whx
                self.Whh -= learning_rate * d_Whh
                self.Why -= learning_rate * d_Why
                self.bh -= learning_rate * d_bh
                self.by -= learning_rate * d_by
            print(f"Epoch: {epoch+1:2}/{epochs:2}, loss: {loss}")
        return self


    def forward_pass(self, x:np.ndarray, index:int = None, get_states=False) -> tuple:
        if not index: index = len(x)
        states:np.ndarray = np.zeros(index)
        x = np.array(x)
        hidden_to_hidden:float = 0
        for i, value in enumerate(x):
            input_to_hidden:float = self.activation(value * self.Whx + hidden_to_hidden + self.bh)
            hidden_to_hidden = input_to_hidden * self.Whh
            states[i] = hidden_to_hidden
        hidden_to_output = input_to_hidden * self.Why + self.by
        
        return (hidden_to_output, None if not get_states else states)
        
        

In [182]:
train_x = [
    list(range(10)),
    list(range(9)),
    list(range(8)),
    list(range(7)),
    list(range(6)),
    list(range(5)),
    list(range(4)),
    list(range(3)),
    list(range(2)),
    list(range(1)),
    list(range(0)),
]
train_y = [
    10,9,7,6,5,4,3,2,1    
]


rnn = RNN(start_random=False).train(train_x, train_y, 0.01, 10)
rnn.forward_pass([1, 2, 3])

IndexError: list index out of range